In [1]:
import numpy as np
import pandas as pd
import my_common as mc
import extract_local_str as els
import scipy.constants as scc

In [3]:
dir_work = ''
%cd dir_work

bc, bond_info, ac, angle_info, dc, dihedral_info = els.read_bond_info('out_anneal500.dat') # specify
bond_idx = bond_info[:,2:].astype(int)-1
result,t = els.read_mutiple_xyz('dump_relax.data') # specify 
bd_neigh = [] ## neighbor list based on bonds
for i in range(len(result[-1][4])):
    tmp = []
    tmp.append(bond_idx[bond_idx[:,0]==i,1])
    tmp.append(bond_idx[bond_idx[:,1]==i,0])
    tmp = np.concatenate(tmp)
    bd_neigh.append(tmp) ##### index in python from 0 

/Volumes/One Touch/DGEBA/react2/melt_21/cooling0d1/compress9x/anneal500_800


In [5]:
# setup SOAP calculation 
from dscribe.descriptors import SOAP 
from ase import Atoms
soap = SOAP(
    species=['C','H','O','N'],
    periodic=True,
    r_cut=4,
    n_max=1,
    l_max=8,
)

In [6]:
# %cd react2/melt_84/cooling0d1/deform9x_800/anneal500
for ia in range(0,1000,100):
    %mkdir ia{ia}

    box = result[ia][0]
    coors = result[ia][4]
    atom_type = result[ia][3]
    
    #
    atom_type_new = np.empty(atom_type.shape,dtype=str)
    atom_type_new[atom_type==1] = 'C'
    atom_type_new[atom_type==2] = 'C'
    atom_type_new[atom_type==3] = 'C'
    atom_type_new[atom_type==4] = 'H'
    atom_type_new[atom_type==5] = 'H'
    atom_type_new[atom_type==6] = 'H'
    atom_type_new[atom_type==7] = 'H'
    atom_type_new[atom_type==8] = 'N'
    atom_type_new[atom_type==9] = 'O'
    atom_type_new[atom_type==10] = 'O'
    atom_type_new[atom_type==11] = 'O'
    case1 = Atoms(atom_type_new,positions=coors,cell=box,pbc=[1,1,1])
    input_deform = soap.create(case1)
    # end 
    
    bond_length = els.BL(box, coors, bond_idx)
    ebond_length = bc[bond_info[:,1].astype(int)-1,2]
    ebond_k = bc[bond_info[:,1].astype(int)-1,1]
    rstrain_bond = (bond_length-ebond_length)/ebond_length
    energy_bond = ebond_k*(bond_length-ebond_length)**2

    idx_stretchbonds = np.squeeze(np.argwhere(rstrain_bond>0.1)) ########### 0.17 strain, 20 energy #############
    idx_stretchatoms = bond_info[idx_stretchbonds,2:].astype(int)-1
    print(idx_stretchbonds.size)

    # distance between bonds 
    # dist_bond_matrix = np.zeros([len(idx_stretchbonds),len(idx_stretchbonds)])
    idx_stretchbonds_screen = idx_stretchbonds.copy()
    for i in range(idx_stretchbonds.size):
        for j in range(i+1,idx_stretchbonds.size):
            dcoor = coors[idx_stretchatoms[i],:] - coors[idx_stretchatoms[j],:]
            dist1 = np.sum(els.minimum_image(dcoor,box)**2,axis=1)**0.5
            dcoor = coors[idx_stretchatoms[i],:] - coors[idx_stretchatoms[j][::-1],:]
            dist2 = np.sum(els.minimum_image(dcoor,box)**2,axis=1)**0.5
            dist_tmp = np.concatenate((dist1,dist2))
            dist_bond = 0.5*np.min(dist_tmp) + 0.5*np.max(dist_tmp)
    #         dist_bond_matrix[i,j] = dist_bond 
    #         dist_bond_matrix[j,i] = dist_bond
            if dist_bond < 8:
                if rstrain_bond[idx_stretchbonds][i] > rstrain_bond[idx_stretchbonds][j]:
                    idx_stretchbonds_screen = idx_stretchbonds_screen[idx_stretchbonds_screen!=idx_stretchbonds[j]]
                else:
                    idx_stretchbonds_screen = idx_stretchbonds_screen[idx_stretchbonds_screen!=idx_stretchbonds[i]]

    idx_stretchbonds = (idx_stretchbonds_screen.copy()).reshape(-1,1)
    idx_stretchatoms = (bond_info[idx_stretchbonds,2:].astype(int)-1).reshape(idx_stretchbonds.size,2)
    print(idx_stretchbonds.size)

    bond_info = bond_info.astype(int)       
    for ib in range(idx_stretchbonds.size):  #range(len(idx_stretchbonds))
        idx_atom = idx_stretchatoms[ib,:]
        n_include,n_inner,atom_type_select,atom_type_add,coors_select,coors_add,idx_atom_interest,idx_inner = \
            els.extr_local_str(idx_atom,box,atom_type,coors,bond_info,bd_neigh,rcut1=8,rcut2=6.5)

        bond_info_inner = []
        for i in range(len(bond_info)): ##### actually include 
            if (bond_idx[i,0] in idx_inner) and (bond_idx[i,1] in idx_inner):
                tmp1 = int(np.argwhere(idx_inner == bond_idx[i,0]))
                tmp2 = int(np.argwhere(idx_inner == bond_idx[i,1]))
                bond_info_inner.append(np.array([bond_info[i,0],bond_info[i,1],tmp1,tmp2,bc[bc[:,0]==bond_info[i,1],1][0],bc[bc[:,0]==bond_info[i,1],2][0],
                                                energy_bond[i],rstrain_bond[i]]))
        bond_info_inner = np.array(bond_info_inner)


        atom_type_include = np.concatenate((atom_type_select,atom_type_add))
        coors_include = np.vstack((coors_select,coors_add))
        !mkdir ia{ia}/elstr{ia}_{ib}_xtb ######## if you change the name, do change the names within 3 lines below
        els.write_xyz('ia{}/elstr{}_{}_xtb/lstr.xyz'.format(ia,ia,ib),n_include,atom_type_include.astype('<U1'),coors_include)
        els.write_orca_input('ia{}/elstr{}_{}_xtb/lstr.inp'.format(ia,ia,ib),'! XTB2 Opt',1,n_include,atom_type_include,coors_include,n_inner) # ! PBEh-3c TIGHTSCF Opt\n%geom maxiter 60 end

        f = open('ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(ia,ia,ib),'w')
        f.write('{} {}\n'.format(idx_atom_interest[0],idx_atom_interest[1]))
        f.write('{} {} #ebond_l, ebond_k\n'.format(ebond_length[idx_stretchbonds][ib],ebond_k[idx_stretchbonds][ib]))
        f.write('{} {} #energy, strain\n\n'.format(energy_bond[idx_stretchbonds][ib],rstrain_bond[idx_stretchbonds][ib]))
        f.write('#bond info inner\n')
        # bond index, bond type, 2 atom index small, 2 index large , k, r0 , energy, rstrain
        for i in range(len(bond_info_inner)): 
            f.write('{0:10d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:10.3f} {7:10.3f} {8:10.3f} {9:10.3f}\n'.format(int(bond_info_inner[i,0]),int(bond_info_inner[i,1]),\
                                                                                  int(bond_info_inner[i,2]),int(bond_info_inner[i,3]),\
                                                                                idx_inner[int(bond_info_inner[i,2])],idx_inner[int(bond_info_inner[i,3])],\
                                                                                 bond_info_inner[i,4],bond_info_inner[i,5],bond_info_inner[i,6],bond_info_inner[i,7]))
        f.close()
        
        soap_bond = []
        for i in range(len(bond_info_inner)): 
            soap_bond.append(np.concatenate((input_deform[idx_inner[int(bond_info_inner[i,2])]],input_deform[idx_inner[int(bond_info_inner[i,3])]])))
        soap_bond = pd.DataFrame(soap_bond)
        soap_bond.to_csv('ia{}/elstr{}_{}_xtb/soap418.csv'.format(ia,ia,ib),index=False,header=False)
        

9
7
43
33
41
37
39
31
47
41
51
41
44
37
32
29
45
33
42
34
